In [2]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json # library to handle JSON files
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
!conda install -c conda-forge beautifulsoup4 --yes
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium # map rendering library

!conda update -n base -c defaults conda --yes
print('Success')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - conda


The following packages will be downloaded:

    package                    |            build
    -----

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')


In [4]:
#table = soup.find_all('table')[0]

#df = pd.read_html(str(table)) 

# or

PostalCodeList1 = []
BoroughList1 = []
NeighborhoodList1 = []

soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        PostalCodeList1.append(cells[0].text.rstrip('\n'))
        BoroughList1.append(cells[1].text.rstrip('\n'))
        NeighborhoodList1.append(cells[2].text.rstrip('\n'))

In [5]:
toronto_df1 = pd.DataFrame({"PostalCode": PostalCodeList1,
                           "Borough": BoroughList1,
                           "Neighborhood": NeighborhoodList1})

toronto_df1.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode           Borough                Neighborhood
0        M1A      Not assigned                            
1        M2A      Not assigned                            
2        M3A        North York                   Parkwoods
3        M4A        North York            Victoria Village
4        M5A  Downtown Toronto  Regent Park / Harbourfront

In [6]:
toronto_df_drop1 = toronto_df1[toronto_df1.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop1.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode           Borough                                  Neighborhood
0        M3A        North York                                     Parkwoods
1        M4A        North York                              Victoria Village
2        M5A  Downtown Toronto                    Regent Park / Harbourfront
3        M6A        North York             Lawrence Manor / Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government

In [7]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_drop1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode      Borough                              Neighborhood
0        M1B  Scarborough                           Malvern / Rouge
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek
2        M1E  Scarborough       Guildwood / Morningside / West Hill
3        M1G  Scarborough                                    Woburn
4        M1H  Scarborough                                 Cedarbrae

In [8]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode      Borough                              Neighborhood
0        M1B  Scarborough                           Malvern / Rouge
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek
2        M1E  Scarborough       Guildwood / Morningside / West Hill
3        M1G  Scarborough                                    Woburn
4        M1H  Scarborough                                 Cedarbrae

In [9]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

<h1>PART 2 - Neighbourhood (Toronto) Project</h1>

In [10]:
#Load Geo Data from CVS
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

In [11]:
toronto_df_geo = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_geo.head()

ImportError: cannot import name 'OrderedDict'

  PostalCode      Borough                              Neighborhood  \
0        M1B  Scarborough                           Malvern / Rouge   
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek   
2        M1E  Scarborough       Guildwood / Morningside / West Hill   
3        M1G  Scarborough                                    Woburn   
4        M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476  

In [12]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df_geo['Latitude'], toronto_df_geo['Longitude'], toronto_df_geo['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
CLIENT_ID = 'PGAINXHLLIKEQPEEM5Z15Q4AJUWWANATC1SSZ5V2S2FRENVF' # your Foursquare ID
CLIENT_SECRET = 'CUBTDRXBXKTMH0R3Q1VP3254CJ4QSUOK2UTNNHAZRPJFEPXC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PGAINXHLLIKEQPEEM5Z15Q4AJUWWANATC1SSZ5V2S2FRENVF
CLIENT_SECRET:CUBTDRXBXKTMH0R3Q1VP3254CJ4QSUOK2UTNNHAZRPJFEPXC


In [18]:
toronto_df_geo.loc[3, 'Neighborhood']

neighborhood_latitude = toronto_df_geo.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df_geo.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df_geo.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Woburn are 43.7709921, -79.21691740000001.


In [16]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url1 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PGAINXHLLIKEQPEEM5Z15Q4AJUWWANATC1SSZ5V2S2FRENVF&client_secret=CUBTDRXBXKTMH0R3Q1VP3254CJ4QSUOK2UTNNHAZRPJFEPXC&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100'

In [17]:
results = requests.get(url1).json()
results

{'meta': {'code': 200, 'requestId': '5e8630671e152c001b586bc7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough Village',
  'headerFullLocation': 'Scarborough Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.768072604500006,
    'lng': -79.18249216787879},
   'sw': {'lat': 43.7590725955, 'lng': -79.1949308321212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c62f34bde1b2d7fec89e370',
       'name': 'G & G Electronics',
       'location': {'address': '4371 kingston road',
        'lat': 43.765309,
        'lng': -79.191537,
        'labeledLatLngs': [{'label': 'display',
          'lat': 4

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


AttributeError: module 'pandas.compat' has no attribute 'string_types'

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(names=toronto_df_geo['Neighborhood'],
                                   latitudes=toronto_df_geo['Latitude'],
                                   longitudes=toronto_df_geo['Longitude']
                                  )

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(2206, 7)


ImportError: cannot import name 'OrderedDict'

                               Neighborhood  Neighborhood Latitude  \
0                           Malvern / Rouge              43.806686   
1  Rouge Hill / Port Union / Highland Creek              43.784535   
2       Guildwood / Morningside / West Hill              43.763573   
3       Guildwood / Morningside / West Hill              43.763573   
4       Guildwood / Morningside / West Hill              43.763573   

   Neighborhood Longitude                  Venue  Venue Latitude  \
0              -79.194353                Wendy’s       43.807448   
1              -79.160497  Royal Canadian Legion       43.782533   
2              -79.188711      G & G Electronics       43.765309   
3              -79.188711       Big Bite Burrito       43.766299   
4              -79.188711  Enterprise Rent-A-Car       43.764076   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.191537     Electronics Store 

In [24]:
toronto_venues.groupby('Neighborhood').count()

ImportError: cannot import name 'OrderedDict'

                                                   Neighborhood Latitude  \
Neighborhood                                                               
Agincourt                                                              4   
Alderwood / Long Branch                                                9   
Bathurst Manor / Wilson Heights / Downsview North                     20   
Bayview Village                                                        4   
Bedford Park / Lawrence Manor East                                    23   
...                                                                  ...   
Wexford / Maryvale                                                     7   
Willowdale                                                            41   
Woburn                                                                 4   
Woodbine Heights                                                      12   
York Mills West                                                        5   

           

In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 267 uniques categories.


In [26]:
## Analyzing Each Neighbourood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

ImportError: cannot import name 'OrderedDict'

   Accessories Store  Afghan Restaurant  Airport  Airport Food Court  \
0                  0                  0        0                   0   
1                  0                  0        0                   0   
2                  0                  0        0                   0   
3                  0                  0        0                   0   
4                  0                  0        0                   0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0             0               0                0                 0   
1             0               0                0                 0   
2             0               0                0                 0   
3             0               0                0                 0   
4             0               0                0                 0   

   American Restaurant  Antique Shop  ...  Train Station  \
0                    0             0  ...              0   
1                    0    

In [27]:
toronto_onehot.shape

(2206, 267)

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

ImportError: cannot import name 'OrderedDict'

                                         Neighborhood  Accessories Store  \
0                                           Agincourt                0.0   
1                             Alderwood / Long Branch                0.0   
2   Bathurst Manor / Wilson Heights / Downsview North                0.0   
3                                     Bayview Village                0.0   
4                  Bedford Park / Lawrence Manor East                0.0   
..                                                ...                ...   
88                                 Wexford / Maryvale                0.0   
89                                         Willowdale                0.0   
90                                             Woburn                0.0   
91                                   Woodbine Heights                0.0   
92                                    York Mills West                0.0   

    Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                 0

In [29]:
toronto_grouped.shape

(93, 267)

In [30]:
num_top_venues = 2

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----Agincourt----
            venue  freq
0          Lounge  0.25
1  Breakfast Spot  0.25


----Alderwood / Long Branch----
         venue  freq
0  Pizza Place  0.22
1     Pharmacy  0.11


----Bathurst Manor / Wilson Heights / Downsview North----
         venue  freq
0         Bank   0.1
1  Coffee Shop   0.1


----Bayview Village----
  venue  freq
0  Café  0.25
1  Bank  0.25


----Bedford Park / Lawrence Manor East----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.09


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05


----Birch Cliff / Cliffside West----
             venue  freq
0  College Stadium  0.25
1             Café  0.25


----Brockton / Parkdale Village / Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09


----Business reply mail Processing CentrE----
           venue  freq
0    Yoga Studio  0.07
1  Auto Workshop  0.07


----CN Tower / King and Spadina / Railway Lands / Harbourfront W

In [31]:
#Rows in decdeing order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

ImportError: cannot import name 'OrderedDict'

                                        Neighborhood  \
0                                          Agincourt   
1                            Alderwood / Long Branch   
2  Bathurst Manor / Wilson Heights / Downsview North   
3                                    Bayview Village   
4                 Bedford Park / Lawrence Manor East   

       1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  
0  Latin American Restaurant        Breakfast Spot                Lounge  
1                Pizza Place                   Gym    Athletics & Sports  
2                       Bank           Coffee Shop    Frozen Yogurt Shop  
3                       Bank    Chinese Restaurant   Japanese Restaurant  
4                Coffee Shop            Restaurant        Sandwich Place  

<h1>Clustering the Neighbourhoods</h1>

In [33]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10].astype(int)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df_geo

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


toronto_merged.head # check the last columns!
toronto_merged.dtypes

PostalCode                object
Borough                   object
Neighborhood              object
Latitude                 float64
Longitude                float64
Cluster Labels           float64
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
dtype: object

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #I could not get color or fill color to work because there are neighborhoods with no top venues due to virus, and I cant covert ClusterLables to in wiht Nulls
        color = 'blue',
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters